In [ ]:
# mount google drive data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# unzip the data_folder zip locally and show any other output
!unzip -q '/content/drive/MyDrive/ipdl_project/emoji/fer2013.zip'

replace test/angry/PrivateTest_10131363.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!pip install keras_preprocessing # Install the missing keras_preprocessing package

# load libraries
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [ ]:
# initialise variables for folders
data_folder="/content/data"
TRAIN_DIR = "/content/train"
TEST_DIR = "/content/test"

In [ ]:
# create a dataframe with image paths and their labels.
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [ ]:
# create a dataframe 'train' with image paths and labels for the training data
train = pd.DataFrame()
train[f"{data_folder}"], train['labels'] = createdataframe(TRAIN_DIR)

disgust completed
sad completed
angry completed
surprise completed
happy completed
fear completed
neutral completed


In [ ]:
print(train)

                                      /content/data   labels
0      /content/train/disgust/Training_46615143.jpg  disgust
1      /content/train/disgust/Training_72272405.jpg  disgust
2      /content/train/disgust/Training_55559187.jpg  disgust
3      /content/train/disgust/Training_11652168.jpg  disgust
4      /content/train/disgust/Training_52126443.jpg  disgust
...                                             ...      ...
28704  /content/train/neutral/Training_65528467.jpg  neutral
28705  /content/train/neutral/Training_20527493.jpg  neutral
28706  /content/train/neutral/Training_53608696.jpg  neutral
28707  /content/train/neutral/Training_55394334.jpg  neutral
28708   /content/train/neutral/Training_5654725.jpg  neutral

[28709 rows x 2 columns]


In [ ]:
# create a dataframe 'test' with image paths and labels for the testing data
test = pd.DataFrame()
test[f"{data_folder}"], test['labels'] = createdataframe(TEST_DIR)

disgust completed
sad completed
angry completed
surprise completed
happy completed
fear completed
neutral completed


In [ ]:
print(test)
print(test[f"{data_folder}"])

                                       /content/data   labels
0     /content/test/disgust/PrivateTest_92933222.jpg  disgust
1       /content/test/disgust/PublicTest_9982221.jpg  disgust
2     /content/test/disgust/PrivateTest_19671520.jpg  disgust
3      /content/test/disgust/PublicTest_16053137.jpg  disgust
4      /content/test/disgust/PublicTest_31237465.jpg  disgust
...                                              ...      ...
7173  /content/test/neutral/PrivateTest_72751389.jpg  neutral
7174  /content/test/neutral/PrivateTest_20073251.jpg  neutral
7175  /content/test/neutral/PrivateTest_53123254.jpg  neutral
7176   /content/test/neutral/PublicTest_26571135.jpg  neutral
7177   /content/test/neutral/PublicTest_37850371.jpg  neutral

[7178 rows x 2 columns]
0       /content/test/disgust/PrivateTest_92933222.jpg
1         /content/test/disgust/PublicTest_9982221.jpg
2       /content/test/disgust/PrivateTest_19671520.jpg
3        /content/test/disgust/PublicTest_16053137.jpg
4        /c

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
# extracts and reshapes image features
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [ ]:
# extract image features for train and test sets
train_features = extract_features(train[f"{data_folder}"])
test_features = extract_features(test[f"{data_folder}"])

  0%|          | 0/28709 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


  0%|          | 0/7178 [00:00<?, ?it/s]

In [ ]:
# normalize pixel values to the range [0, 1]
x_train = train_features/255.0
x_test = test_features/255.0

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# fit label encoder on training labels
le = LabelEncoder()
le.fit(train['labels'])

LabelEncoder()

In [ ]:
# encode labels for train and test sets
y_train = le.transform(train['labels'])
y_test = le.transform(test['labels'])

In [ ]:
# one-hot encode labels
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [ ]:
# define the cnn model architecture

model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# compile the model
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [ ]:
# train the model
history = model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test))

Epoch 1/100
 17/225 ━━━━━━━━━━━━━━━━━━━━ 21:29 6s/step - accuracy: 0.2223 - loss: 1.8718

KeyboardInterrupt: 

In [ ]:
# save the trained model
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [ ]:
from keras.models import model_from_json

In [ ]:
# load the saved model
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [ ]:
# predict and display a random image

import random
import os
from keras_preprocessing.image import load_img
import numpy as np
import matplotlib.pyplot as plt

label = ['angry','disgust','fear','happy','neutral','sad','surprise']

def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

# Randomly select train or test folder
folder = random.choice([TRAIN_DIR, TEST_DIR])

# Randomly select emotion folder
emotion_folder = random.choice(label)

# Randomly select an image from the chosen folder
image_path = random.choice(os.listdir(os.path.join(folder, emotion_folder)))
image = os.path.join(folder, emotion_folder, image_path)

# Perform prediction
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]

# Load and display image using matplotlib.pyplot
img = load_img(image, grayscale=True)  # Load image for display
img = np.array(img)  # Convert to NumPy array

plt.imshow(img, cmap='gray')  # Display image in grayscale
plt.title(f"Original: {emotion_folder}, Prediction: {pred_label}")  # Set title
plt.axis('off')  # Hide axes
plt.show()

In [ ]:
# accuracy values
train_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

In [ ]:
# print accuracy percentage
final_train_accuracy = train_accuracy[-1] * 100
final_validation_accuracy = validation_accuracy[-1] * 100

print(f"Final Training Accuracy: {final_train_accuracy:.2f}%")
print(f"Final Validation Accuracy: {final_validation_accuracy:.2f}%")

In [ ]:
# accuracy graph
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Acc','Val'], loc = 'upper left')
plt.show()